In [1]:
from math import exp
# Calculate neuron activation (net) for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation
 
# Transfer neuron activation to sigmoid function 
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))
 
# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs
 
# test forward propagation
network = [[{'weights': [0.2,0.4,-0.5,-0.4]},{'weights':[-0.3,0.1,0.2,0.2]}], [{'weights': [-0.3,-0.2,0.1]}]]
row = [1, 0, 1]
output = forward_propagate(network, row)
print(output)

[0.47388889882398544]


In [2]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output) # ok(1-ok) or oh(1-oh)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        
        if i != len(network)-1: # Check for hidden layer or not
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta']) # summation of Errk * Whk
                errors.append(error)
        else:                           # output layer
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output']) # (Tk-ok)
        
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 



In [3]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
 

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
   
    print("\n Network Training Begins:\n")
    
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            #expected = [0 for i in range(n_outputs)]
            expected = [1]
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    
    print("\n Network Training Ends:\n")
 

In [5]:
#Test training backprop algorithm
dataset=[[1,0,1,1]]
print("\n The input Data Set :\n",dataset)
n_inputs = len(dataset[0]) - 1
print("\n Number of Inputs :\n",n_inputs)
n_outputs = len(set([row[-1] for row in dataset]))
print("\n Number of Outputs :\n",n_outputs)

#Network Initialization
network = [[{'weights': [0.2,0.4,-0.5,-0.4]},{'weights':[-0.3,0.1,0.2,0.2]}], [{'weights': [-0.3,-0.2,0.1]}]]

# Training the Network
train_network(network, dataset, 0.90, 1, n_outputs)


print("\n Final Neural Network :")
    
i= 1
for layer in network:
    j=1
    for sub in layer:
        print("\n Layer[%d] Node[%d]:\n" %(i,j),sub)
        j=j+1
    i=i+1


 The input Data Set :
 [[1, 0, 1, 1]]

 Number of Inputs :
 3

 Number of Outputs :
 1

 Network Training Begins:

>epoch=0, lrate=0.900, error=0.277

 Network Training Ends:


 Final Neural Network :

 Layer[1] Node[1]:
 {'weights': [0.19214789423111006, 0.4, -0.50785210576889, -0.40785210576888997], 'output': 0.3318122278318339, 'delta': -0.008724561965433263}

 Layer[1] Node[2]:
 {'weights': [-0.30588787655775207, 0.1, 0.19411212344224793, 0.19411212344224793], 'output': 0.52497918747894, 'delta': -0.006542085064168994}

 Layer[2] Node[1]:
 {'weights': [-0.26082884634154524, -0.13802506750700472, 0.21805217039291008], 'output': 0.47388889882398544, 'delta': 0.1311690782143445}
